In [ ]:
pip install  pmdarima

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 5.0 MB/s 
     |████████████████████████████████| 9.8 MB 32.5 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [ ]:
import pandas as pd
import numpy as np
import warnings
from sklearn import metrics as mt
from pmdarima.arima import auto_arima

In [ ]:
y_train = pd.read_csv('y_train_complete.csv', index_col=['date'], parse_dates=['date'])
y_test = pd.read_csv('y_test.csv', index_col=['date'], parse_dates=['date'])

In [ ]:
y_test.index.name = 'Date'
y_test.rename(columns={"reference_evapotranspiration": "REFERENCE_EVAPOTRANSPIRATION"});

In [ ]:
reference_evapotranspiration = pd.concat([y_train, y_test])

In [ ]:
arima_model = auto_arima(reference_evapotranspiration.reference_evapotranspiration,
                         start_p=1, start_q=1, max_p=10, max_q=10, m=12,
                          start_P=0, seasonal=False, d=1, D=1, trace=True,
                          error_action='ignore',  # don't want to know if an order does not work
                          suppress_warnings=True,  # don't want convergence warnings
                          stepwise=True)  # set to stepwise

/usr/local/lib/python3.7/dist-packages/pmdarima/arima/_validation.py:62: UserWarning: m (12) set for non-seasonal fit. Setting to 0
  warnings.warn("m (%i) set for non-seasonal fit. Setting to 0" % m)


Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-2122.503, Time=0.38 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-2071.188, Time=0.07 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-2081.174, Time=0.21 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-2092.745, Time=0.35 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-2073.186, Time=0.02 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=-2115.242, Time=0.62 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=-2117.501, Time=0.31 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=-2118.469, Time=0.40 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=-2097.809, Time=0.12 sec
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=-2119.972, Time=0.94 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=-2124.502, Time=0.16 sec
 ARIMA(0,1,1)(0,0,0)[0]             : AIC=-2092.483, Time=0.03 sec
 ARIMA(1,1,0)(0,0,0)[0]             : AIC=-2083.172, Time=0.04 sec
 ARIMA(2,1,1)(0,0,0)[0]             : AIC=-2123.705, Time=0.30 sec
 ARIMA(1,1,2)(0,0,0

In [ ]:
arima_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                  334
Model:               SARIMAX(1, 1, 1)   Log Likelihood                1065.251
Date:                Sat, 04 Jun 2022   AIC                          -2124.502
Time:                        05:41:20   BIC                          -2113.077
Sample:                             0   HQIC                         -2119.946
                                - 334                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.5091      0.071      7.171      0.000       0.370       0.648
ma.L1         -0.8768      0.046    -19.134      0.000      -0.967      -0.787
sigma2      9.722e-05   6.24e-06     15.577      0.000     8.5e-05       0.000
===================================================================================
Ljung-Box (L1) (Q):                   0.32   Jarque-Bera (JB):                18.09
Prob(Q):                              0.57   Prob(JB):                         0.00
Heteroskedasticity (H):               1.06   Skew:                            -0.25
Prob(H) (two-sided):                  0.76   Kurtosis:                         4.03
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [ ]:
arima_model.fit(y_train.reference_evapotranspiration)

ARIMA(order=(1, 1, 1), scoring_args={}, suppress_warnings=True,
      with_intercept=False)

In [ ]:
y_pred = arima_model.predict(n_periods=len(y_test))

print('RMSE = {}\nMAE  = {}\n'.format(
    np.sqrt(mt.mean_squared_error(y_test.reference_evapotranspiration, y_pred)), 
    mt.mean_absolute_error(y_test.reference_evapotranspiration, y_pred)))

RMSE = 0.018997837399931612
MAE  = 0.016712288558646364

